Telepítsük le a fontos könyvtárakat

In [6]:
!pip install newspaper3k
!pip install lxml_html_clean
!pip install datasets

A letelepített könyvtárakból importáljuk be a szükséges részeket

In [7]:
#======================
#Adathalmaz létrehozása
#======================

from newspaper import Article
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random
from tqdm import tqdm
import os
import datetime
import requests
from google.colab import files


# Az Origoról két lépésben szedem le a cikkeket:
1. Összegyűjtöm dátum alapján a linkeket
2. A linkek alapján összegyűjtöm a cikkeket

In [8]:
# Importáljuk a datetime modult, amely segít kezelni a dátumokat


# Létrehozunk egy üres listát az url-ek tárolására
urls = []

# Létrehozunk egy datetime objektumot 2023. január 1-jével
date = datetime.date(2025, 3, 19)

# Létrehozunk egy másik datetime objektumot 2024. január 1-jével
end_date = datetime.date(2025, 4, 20)

# Végigmegyünk a dátumokon egy napos lépésekkel, amíg el nem érjük a végdátumot
while date < end_date:
    # Kinyerjük a dátum évét, hónapját és napját
    year = date.year
    month = date.month
    day = date.day
    # Összerakjuk az url-t a dátummal
    url = f"https://www.origo.hu/hirarchivum/{year}/{month}/{day}"
    # Hozzáadjuk az url-t a listához
    urls.append(url)
    # Növeljük a dátumot egy nappal
    date += datetime.timedelta(days=1)

# Kiírjuk az url-ek listáját
print(urls)

['https://www.origo.hu/hirarchivum/2025/3/19', 'https://www.origo.hu/hirarchivum/2025/3/20', 'https://www.origo.hu/hirarchivum/2025/3/21', 'https://www.origo.hu/hirarchivum/2025/3/22', 'https://www.origo.hu/hirarchivum/2025/3/23', 'https://www.origo.hu/hirarchivum/2025/3/24', 'https://www.origo.hu/hirarchivum/2025/3/25', 'https://www.origo.hu/hirarchivum/2025/3/26', 'https://www.origo.hu/hirarchivum/2025/3/27', 'https://www.origo.hu/hirarchivum/2025/3/28', 'https://www.origo.hu/hirarchivum/2025/3/29', 'https://www.origo.hu/hirarchivum/2025/3/30', 'https://www.origo.hu/hirarchivum/2025/3/31', 'https://www.origo.hu/hirarchivum/2025/4/1', 'https://www.origo.hu/hirarchivum/2025/4/2', 'https://www.origo.hu/hirarchivum/2025/4/3', 'https://www.origo.hu/hirarchivum/2025/4/4', 'https://www.origo.hu/hirarchivum/2025/4/5', 'https://www.origo.hu/hirarchivum/2025/4/6', 'https://www.origo.hu/hirarchivum/2025/4/7', 'https://www.origo.hu/hirarchivum/2025/4/8', 'https://www.origo.hu/hirarchivum/2025/4/

In [9]:
# Egy lista a magyar politikai weboldalakról, amelyekről híreket akarsz kaparni

# Egy üres lista a hírek tárolására
news = []
neutral_news = []


# Végigmegyünk a weboldalak listáján
for website in urls:

    # Lekérjük a weboldal tartalmát
    response = requests.get(website)
    # Létrehozunk egy BeautifulSoup objektumot a HTML elemzéséhez
    soup = BeautifulSoup(response.text, "html.parser")
    # Megkeressük az összes hírcímet tartalmazó HTML elemet (ez változhat az oldaltól függően)
    links = soup.find_all("a", href=True)
    # Végigmegyünk a hírcímeken
    for link in links:
      title = "Nincs cím"  # Alapértelmezett cím
      title_tag = link.find(class_="categorized-article-container-title")
      if title_tag is not None:
          title = title_tag.text
      href = link["href"]

      # Kinyerjük a hírcím szövegét
      href = link["href"]
      if href.startswith("/itthon"):
          # Létrehozunk egy Article objektumot a linkkel
          article = Article(f'https://origo.hu'+href)
          # Letöltjük és elemezzük a cikk HTML-jét
          try:
            article.download()
            article.parse()

          # Kinyerjük a cikk szövegét
            text = article.text
          # Hozzáadjuk a hírt a listához egy szótár formájában
            news.append({"title": title, "link": f'https://origo.hu'+href, "text": text, "date": published_date})

          except:
            print("Hiba történt egy cikk letöltése vagy elemzése közben")
      elif href.startswith('/auto/') or href.startswith('/tudomany/') or href.startswith('/sport/'):
          # Létrehozunk egy Article objektumot a linkkel
          article = Article(f'https://origo.hu'+href)
          # Letöltjük és elemezzük a cikk HTML-jét
          article.download()
          article.parse()
          # Kinyerjük a cikk szövegét
          text = article.text
          # dátum
          published_date = article.publish_date
          # Hozzáadjuk a hírt a listához egy szótár formájában
          neutral_news.append({"title": title, "link": f'https://origo.hu'+href, "text": text, "date": published_date})
    print(f"news from {website}: {len(news)}")



news from https://www.origo.hu/hirarchivum/2025/3/19: 22
news from https://www.origo.hu/hirarchivum/2025/3/20: 49
news from https://www.origo.hu/hirarchivum/2025/3/21: 71
news from https://www.origo.hu/hirarchivum/2025/3/22: 90
news from https://www.origo.hu/hirarchivum/2025/3/23: 110
news from https://www.origo.hu/hirarchivum/2025/3/24: 138
news from https://www.origo.hu/hirarchivum/2025/3/25: 159
news from https://www.origo.hu/hirarchivum/2025/3/26: 183
news from https://www.origo.hu/hirarchivum/2025/3/27: 206
news from https://www.origo.hu/hirarchivum/2025/3/28: 233
news from https://www.origo.hu/hirarchivum/2025/3/29: 254
news from https://www.origo.hu/hirarchivum/2025/3/30: 273
news from https://www.origo.hu/hirarchivum/2025/3/31: 294
news from https://www.origo.hu/hirarchivum/2025/4/1: 316
news from https://www.origo.hu/hirarchivum/2025/4/2: 338
news from https://www.origo.hu/hirarchivum/2025/4/3: 360
news from https://www.origo.hu/hirarchivum/2025/4/4: 391
news from https://www.

# A telexre más módszert alkalmaztam, de a gondolatmenet megegyezik

In [10]:
def scrape_telex_page(url):
    """Scrape articles from a single Telex archive page."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return [], []

    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('a', class_="list__item__title", href=True)

    pro_gov_news = []

    for article_tag in articles:
        try:
            href = article_tag['href']
            if not href.startswith('/'):
                continue

            link = f'https://telex.hu{href}'
            title = article_tag.find(class_="hasHighlight").text.strip()

            # Download article using newspaper3k
            article = Article(link)
            article.download()
            article.parse()
            text = article.text
            published_date = article.publish_date
            article_data = {"title": title, "link": link, "text": text, "date": published_date}

            if '/belfold/' in href:
                pro_gov_news.append(article_data)

            # Random sleep to avoid being blocked
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error processing article: {e}")

    return pro_gov_news




# Settings
start_page = 1
end_page = 219  # You can increase this as needed

# fájl létrehozása, amibe később rakjuk az összes adatot
!mkdir Adathalmaz

# Storage
all_news = []

# Main scraping loop
for page in tqdm(range(start_page, end_page + 1)):
    url = f"https://telex.hu/archivum?oldal={page}"
    news_batch = scrape_telex_page(url)
    all_news.extend(news_batch)

    # Periodically save progress
    if page % 10 == 0 or page == end_page:
        pd.DataFrame(all_news).to_csv(os.path.join("Adathalmaz/telex_news.csv"), index=False)

print(f"Scraping finished! Total pro-gov: {len(all_news)}")

 19%|█▊        | 41/219 [15:00<1:08:50, 23.21s/it]

Error processing article: Article `download()` failed with 500 Server Error: Internal Server Error for url: https://telex.hu/sport/2025/04/19/bode-daniel-golt-lott-500-nb-i-es-meccsen on URL https://telex.hu/sport/2025/04/19/bode-daniel-golt-lott-500-nb-i-es-meccsen


 21%|██        | 45/219 [16:31<1:07:00, 23.11s/it]

Error processing article: Article `download()` failed with 500 Server Error: Internal Server Error for url: https://telex.hu/kult/2025/04/19/horn-gabor-es-horn-gyorgy-alinda-oktatas on URL https://telex.hu/kult/2025/04/19/horn-gabor-es-horn-gyorgy-alinda-oktatas


 24%|██▍       | 53/219 [19:32<1:02:07, 22.46s/it]

...

Error processing article: Article `download()` failed with 500 Server Error: Internal Server Error for url: https://telex.hu/belfold/2025/03/26/orban-viktor-gyuloletbeszed-ugyeszseg-elutasitas on URL https://telex.hu/belfold/2025/03/26/orban-viktor-gyuloletbeszed-ugyeszseg-elutasitas


 96%|█████████▌| 210/219 [1:20:33<03:40, 24.53s/it]

Error processing article: Article `download()` failed with 500 Server Error: Internal Server Error for url: https://telex.hu/gazdasag/2025/03/26/orban-viktor-kormanyules-falu-tamogatas-atm on URL https://telex.hu/gazdasag/2025/03/26/orban-viktor-kormanyules-falu-tamogatas-atm


100%|██████████| 219/219 [1:24:04<00:00, 23.03s/it]

Scraping finished! Total pro-gov: 675


# Teljes adatbázis létrehozása

Összeszedtem 3 különböző DataFramebe a cikkeket, hogy könnyebb legyen a cimkézés

In [26]:
origo_df = pd.DataFrame(news)
neutral_df = pd.DataFrame(neutral_news)
telex_df = pd.DataFrame(all_news)

origo_df.to_csv("Adathalmaz/origo_news.csv", index=False)
neutral_df.to_csv("Adathalmaz/neutral_news.csv", index=False)
telex_df.to_csv("Adathalmaz/telex_news.csv", index=False)


print("neutral:",len(neutral_news))
print("origo:",len(news))
print("telex:",len(all_news))

neutral: 934
origo: 679
telex: 675


Címkézem és egyesítem az adatokat

In [27]:

# Add labels
telex_df['label'] = 'telex'
neutral_df['label'] = 'neutral'
origo_df['label'] = 'origo'

# Concatenate the dataframes
combined_df = pd.concat([neutral_df, telex_df,origo_df], ignore_index=True)
combined_df.to_csv("Adathalmaz/combined.csv", index=False)

# Now combined_df contains both dataframes with their respective labels
combined_df.head()

,title,link,text,date,label
0,Nincs cím,https://origo.hu/sport/rovat/sport-csapat,A Fradi 14 év után már biztosan érmes kézilabd...,None,neutral
1,Nincs cím,https://origo.hu/sport/rovat/sport-egyeni,Hatodik olimpiájára készül a legendás magyar s...,None,neutral
2,Nincs cím,https://origo.hu/sport/rovat/sport-futball,"Döbbenetes az a hiszti, amit a Real Madrid műv...",None,neutral
3,Nincs cím,https://origo.hu/sport/rovat/f1,Molnár Martin a bajnoki címért száll harcba a ...,None,neutral
4,Nincs cím,https://origo.hu/sport/rovat/sport-laza,Szuperdögös barna bombázóért készül elhagyni f...,None,neutral


In [29]:
origo = pd.read_csv('Adathalmaz/origo_news.csv')
origo['target'] = 0
telex = pd.read_csv('Adathalmaz/telex_news.csv')
telex['target'] = 1
neutral = pd.read_csv('Adathalmaz/neutral_news.csv')
neutral['target'] = 2

Létrehozok egy olyan adathalmazt is, ahol csak a nyers szöveg van és a címke

In [30]:
allnews = pd.concat([origo, telex, neutral])
allnews = allnews[['target','text']]
allnews = allnews.dropna()
allnews.to_csv("Adathalmaz/allnews.csv", index=False)


In [32]:
from zipfile import ZipFile
import os

file_path = []

for root, directories, files in os.walk('Adathalmaz'):
    for filename in files:
        file_path.append(os.path.join(root, filename))

with ZipFile('Adathalmaz.zip','w') as zip:
   # writing each file one by one
   for file in file_path:
       zip.write(file)

